# Unstructured Entity Recognition

In [1]:
# The code was removed by DSX for sharing.

In [ ]:
from io import BytesIO  

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print(resp2)

In [ ]:
import io
import requests
import zipfile
import os

url=requests.get('https://www.techgig.com/files/DataScienceFullData/11782_100123/TechGig-DataSet.zip')
data = zipfile.ZipFile(io.BytesIO(url.content))
data.extractall()

In [2]:
path = os.getcwd()
os.chdir(path+'/TechGig DataSet')
print os.listdir(os.getcwd())

['MNB.csv', 'pred2.csv', 'train.csv', 'feat.pickle', 'test_r.csv', 'pred.csv', 'GBN.csv', 'sample-submission.csv', 'important', 'test.csv', 'pred1.csv', 'train_r.csv']


In [3]:
import pandas as pd
import numpy as np
import gc

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
print train.shape
print test.shape

(9850, 3)
(21070, 2)


In [7]:
train.head()

,id,StringToExtract,description
0,3,ddlsql144,fotrscomi.srsrelvqe.2008.agentjob.jobduration ...
1,7,ddlsql43,a sql job failed to complete successfully. sql...
2,13,ddlsql43,a sql job failed to complete successfully. sql...
3,19,ddlsql43,a sql job failed to complete successfully. sql...
4,22,fra-sql-03,srsrelvqeagent - a sql job failed to complete ...


In [8]:
test.head()

,id,description
0,1,request 1 - http://frapub2.intranet.ecnahcdrof...
1,2,auth critical alert detected detected entry: ...
2,4,a sql job failed to complete successfully. sql...
3,5,oionr advanced alert alert me when an interfac...
4,6,a sql job failed to complete successfully. sql...


# Understanding Data

In [ ]:
len(np.unique(train.StringToExtract))

There are 1311 unique lables in training set.Out of Which top five labled in dataset  most are

ddlsql43                                    3817

esc-ev-11                                    442

nycpub                                       115

ddlsql37                                     109

ddl-mocsrds-01                               101

# Processing Text
0. Combine train and text files to procress data at once

1. Make description to lowercase 

2. Remove urls from the description

3. Removing date and time from description


In [9]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize ,sent_tokenize

In [10]:
#combine train and test dataset for preprocessing data
test = test.assign(StringToExtract='NA')
df=train.append(test,ignore_index=True)

In [11]:
#Remove url and date and time,lower case
stop = nltk.corpus.stopwords
listofstops=set(stop.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
postem=PorterStemmer()

In [ ]:
def lower_set(x):
    return x.decode('utf-8').lower()
def remove_url(x):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', x)
def remove_date_time(x):
    td=re.compile(r'\d{1,2}[-/]\d{1,2}[-/]\d{4}|\d{4}[-/]\d{1,2}[-/]\d{1,2}|\d{1,2}[:]\d{1,2}[:]\d{1,2}[\s][paPA][mM]|\d{1,2}[:]\d{1,2}[:]\d{1,2}|\d{1,2}[:]\d{1,2}')#10-3-2017
    return re.sub(td,'',x)
def del_stopwords(x):
    return ' '.join([i for i in word_tokenize(x) if i not in listofstops])
def stem_doc(x):
    red_text = [postem.stem(word.strip()) for word in x.split(" ") if word.strip()!='']
    return ' '.join(red_text)
def clean_words(words):
    if words:
        words=words.replace('.',' ')
        words = words.replace('\t',' ')
        words = words.replace('\r',' ')
        words = words.replace('\n',' ')
        words = words.replace(',',' ')
        words = words.replace(':',' ')
        words = words.replace(';',' ')
        words = words.replace('=',' ')
        #words = words.replace('\x92','') # apostrophe encoding
        #words = words.replace('\x08','\\b') # \b is being treated as backspace
        #words = ''.join([i for i in words if not i.isdigit()])
        words = words.replace('_',' ')
        words = words.replace('(',' ')
        words = words.replace(')',' ')
        words = words.replace('+',' ')
        #words = words.replace('-',' ')
        words = words.replace('`',' ')
        words = words.replace('\'',' ')
        #words = words.replace('.',' ')
        words = words.replace('#',' ')
        words = words.replace('/',' ')
        words = words.replace('_',' ')
        words = words.replace('"',' ')
        words = words.replace('$',' ')
        words = words.replace('}',' ')
        words = words.replace('{',' ')
    return words.strip()
def final_clean(x):
    return ' '.join([i.strip() for i in x.split()])
##REmoving CD i.e  	Cardinal number 
def remove_cd(sent):
    tag=pos_tag(sent.split())
    return ' '.join([i for i,j in tag if j!='CD'])
##Removing words like na 225 internal as they occur in almost all data record
lists=['na','internal','225']
def remove_internal_na(sent):
    return ' '.join([i for i in sent.split(' ') if i not in lists])

In [ ]:
df = df.assign(description=df.description.apply(lambda x: lower_set(x)))
df = df.assign(description=df.description.apply(lambda x: remove_url(x)))
df=df.assign(description=df.description.apply(lambda x: remove_date_time(x)))
df=df.assign(description=df.description.apply(lambda x: del_stopwords(x)))
df=df.assign(description=df.description.apply(lambda x: stem_doc(x)))
df=df.assign(description=df.description.apply(lambda x:clean_words(x)))
df.description = df.description.apply(lambda x:x.encode('utf-8'))
#df=df.assign(description=df.description.apply(lambda x:remove_cd(x)))
df=df.assign(description=df.description.apply(lambda x:final_clean(x)))
df=df.assign(description=df.description.apply(lambda x:remove_internal_na(x)))

Try to use different feature extraction like TfidfVectorizer,CountVectorizer,HashingVectorizer
out of which CountVectorizer has best result

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
#Getting words from the train description to make wordvector 
five_g=CountVectorizer(vocabulary=np.unique(train.StringToExtract))
five_g.fit(df.description)
somenew=five_g.fit_transform(df.description)

In [ ]:
#seperating train and test dataset for prediction
df=pd.DataFrame(somenew.toarray())
df.columns=five_g.get_feature_names()
newtrain=df.loc[:len(train)-1]
newtest=df.loc[len(train):]

In [ ]:
#The given dataset is highly imbalanced balancing the subsample is one approach
def balanced_subsample(y, size=None)
    subsample = []
    if size is None:
        n_smp = y.value_counts().min()
    else:
        n_smp = int(size / len(y.value_counts().index))

    for label in y.value_counts().index:
        samples = y[y == label].index.values
        index_range = range(samples.shape[0])
        indexes = np.random.choice(index_range, size=n_smp, replace=False)
        subsample += samples[indexes].tolist()
    return subsample

In [ ]:
newtrain=train.loc[balanced_subsample(train.StringToExtract)]
balanced_train=pd.concat([train]*200)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
model=gnb.fit(newtrain,train.StringToExtract)
pred=model.predict(newtest)

In [ ]:
sol=pd.DataFrame(pred)
sol.to_csv('pred.csv')
finalsol =pd.read_csv('sample-submission.csv')

finalsol=finalsol.assign(StringToExtract=pred)

finalsol.to_csv('GBN.csv',quoting=False,index=False)

The naive bayes has the best accuracy when compared to other model(rf,xgboost,svm) with accuracy around 90.8

###### Below are some of the techniques which can improve the model but in this case these were not helpful

In [ ]:
processed_train=df[df.StringToExtract !='NA']
processed_test=df[df.StringToExtract =='NA']

In [ ]:
valuescount=processed_train.StringToExtract.value_counts()

replicate_subset_list=processed_train.StringToExtract.value_counts()[(processed_train.StringToExtract.value_counts()==1)].index.tolist()

replicate_subset=processed_train[processed_train.StringToExtract.apply(lambda x: x in replicate_subset_list)]

replicated_set=processed_train.loc[np.repeat(replicate_subset.index.values,repeats=5)]

In [ ]:
from nltk.probability import FreqDist

feq=FreqDist()

doc=[]

df.description.apply(lambda x: doc.append(x))

fdist = FreqDist(word.lower() for word in word_tokenize(' '.join(doc)))

lists=[i for i,j in fdist.most_common()]##Removing most common words like net ,internal and so on 
##one label is missing in those words key role missing ddl43

In [ ]:
s=[ i for i in lists if i not in lables]

In [ ]:
def search(sent):
    sent=' '.join(sent.split('.'))
    lis=[i for i in sent.split() if i  in lables]
    return ' '.join(lis)

In [ ]:
df.description=df.description.apply(lambda x: search(x))

In [ ]:
## MAKE NEW CLASSES BASED ON NET,COM BECASE THERE ARESOME CLASSEES THAT ARE MISLABLED
train[train.StringToExtract=='knowwho'].description.loc[2972]

In [ ]:
lables=np.unique(train.StringToExtract)

In [ ]:
newprocessedtrain=processed_train.append(replicated_set,ignore_index=True)

newprocessedtrain.description=newprocessedtrain.description.apply(lambda x: ' '.join([ i for i in x.split() if i not in lists]))

In [ ]:
totaldf=newprocessedtrain.description.append(processed_test.description)

In [ ]:
#tfv=TfidfVectorizer(max_features=500)
tfv=CountVectorizer(max_features=3000)
tfv.fit(newprocessedtrain.description,newprocessedtrain.StringToExtract)

feat_ext=tfv.fit_transform(totaldf)

newfeat=feat_ext.toarray()

newtrain=newfeat[:12320]
newtest=newfeat[12320:]

In [ ]:
newdf =pd.DataFrame(newfeat)
newdf=newdf.assign(StringToExtract=df.StringToExtract)

In [ ]:
newtrain=newdf.loc[newdf.StringToExtract != 'NA']
newtest=newdf.loc[newdf.StringToExtract == 'NA']
newtest.drop(['StringToExtract'],axis=1,inplace=True)

newtrain.drop(['StringToExtract'],axis=1,inplace=True)

In [ ]:
import re

net=re.compile(pattern=r"""[-\w\.]*\.net""")
com=re.compile(pattern=r"""([[\w\.-]*]*\.com)""")
dash=re.compile(pattern=r"""[\w-]*""")

def find_net_com(sent):
    lis=[]
    lis=net.findall(sent)    
    lis.append(' '.join(com.findall(sent)))
    lis.append(' '.join(dash.findall(sent)))
    return ' '.join(lis)

In [ ]:
newtrain=train.description.apply(lambda x:find_net_com(x))
newtest=test.description.apply(lambda x:find_net_com(x))

In [ ]:
df=newtrain.append(newtest)

final =pd.DataFrame(feat.toarray())

finaltrain =final.loc[:len(train)-1]
finaltest=final.loc[len(train):]

In [ ]:
def chunking(sent):
    
    chunkGram = r"""Chunk: {<NN|VBP|JJ><JJ><JJ><JJ>}
                    Chunk:{<JJ>\.<NN>\.<NN>\.<NN>}
                    Chunk:  {<JJ><NN><JJ><JJ>}  
                    Chunk: {<CD>\.<CD>\.<CD>\.<CD>}
                    Chunk : {<JJ>\.<IN><NN>\.<NN>\.<NN>}"""
    chunkParser = nltk.RegexpParser(chunkGram)
    chunked = chunkParser.parse(pos_tag(sent.split()))
    res =''
    for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Chunk'):
                    res+= str(subtree)
    return res